🚗🔌 Projet Open Data University : Infrastructures de Recharge des Véhicules Électriques

📘 Notebook de démarrage - Analyse temporelle et spatiale

In [ ]:
# 📦 1. IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import plotly.express as px
from datetime import datetime

In [ ]:
# 📁 2. CHARGEMENT DES DONNÉES
# ⚠️ À ADAPTER : chemins vers vos fichiers CSV
# Exemple : bornes_df = pd.read_csv("data/irve.csv")

bornes_df = pd.read_csv("data/irve.csv")  # Données IRVE
population_df = pd.read_csv("data/population.csv")  # Données population INSEE
vehicules_df = pd.read_csv("data/vehicules.csv")  # Données parc de véhicules
trafic_df = pd.read_csv("data/trafic_tmja.csv")  # Données trafic routier

In [ ]:
# 🧼 3. NETTOYAGE & HARMONISATION
# ✅ Uniformisation des codes communes
# ⚠️ Vérifiez les noms des colonnes dans vos fichiers

bornes_df["code_commune"] = bornes_df["code_commune_insee"].astype(str)
population_df["code_commune"] = population_df["CODGEO"].astype(str)
vehicules_df["code_commune"] = vehicules_df["CODGEO"].astype(str)

# Conversion des dates si nécessaire
bornes_df["date_mise_en_service"] = pd.to_datetime(bornes_df["date_mise_en_service"], errors='coerce')


In [ ]:
# 🔗 4. FUSION DES DONNÉES
# Fusion sur le code commune (join left pour garder IRVE comme base principale)

merged_df = bornes_df.merge(population_df, on="code_commune", how="left")
merged_df = merged_df.merge(vehicules_df, on="code_commune", how="left")


In [ ]:
# 📈 5. ANALYSE TEMPORELLE : évolution des bornes
bornes_df["annee_mise_en_service"] = bornes_df["date_mise_en_service"].dt.year
bornes_par_annee = bornes_df.groupby("annee_mise_en_service").agg({"id_pdc_local": "count"}).reset_index()
bornes_par_annee.columns = ["Année", "Nombre de bornes"]

plt.figure(figsize=(10, 5))
sns.barplot(data=bornes_par_annee, x="Année", y="Nombre de bornes", color="skyblue")
plt.title("Évolution annuelle des bornes mises en service")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 🗺️ 6. ANALYSE SPATIALE SIMPLE : nombre de bornes par commune
bornes_par_commune = bornes_df.groupby("code_commune").agg({"id_pdc_local": "count"}).reset_index()
bornes_par_commune.columns = ["code_commune", "nb_bornes"]

# Fusion avec population
communes_stats = bornes_par_commune.merge(population_df, on="code_commune", how="left")
communes_stats["bornes_pour_10000_hab"] = communes_stats["nb_bornes"] / (communes_stats["population"] / 10000)


In [ ]:
# 🌍 7. CARTE CHOROPLÈTHE : bornes pour 10 000 habitants
# ⚠️ À adapter selon disponibilité des géométries (fichier GeoJSON ou shapefile)

# Exemple d’import d’un fichier geojson des communes
# geo_df = gpd.read_file("data/communes.geojson")
# geo_df = geo_df.merge(communes_stats, on="code_commune")
# geo_df.plot(column="bornes_pour_10000_hab", cmap="viridis", legend=True)


In [ ]:
# 🧠 8. SCORE DE PRIORISATION (brouillon)
# Normalisation (min-max) et score composite (exemple simplifié)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
communes_stats[["pop_norm"]] = scaler.fit_transform(communes_stats[["population"]])
communes_stats[["bornes_norm"]] = scaler.fit_transform(communes_stats[["nb_bornes"]])

communes_stats["score_besoin"] = communes_stats["pop_norm"] - communes_stats["bornes_norm"]

In [ ]:
# 🧾 9. EXPORT DES DONNÉES
communes_stats.to_csv("data/communes_scores.csv", index=False)

In [ ]:
# 📌 À FAIRE ensuite :
# - Ajouter TMJA + analyse par axe routier
# - Projection 2030 (régression)
# - Cartes interactives (Plotly / Folium)
# - Rédiger recommandations
# - Nettoyer le notebook final